In [66]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Activation, RepeatVector, Dropout, Dense
from keras.layers import LSTM
from keras.layers.wrappers import TimeDistributed
import numpy as np
import os 

from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [67]:
# Function to convert integer input to vectors for RNN using one hot encoding

def one_hot_encode(X,seq_len, vocab_size):
    x = np.zeros((len(X),seq_len, vocab_size), dtype=np.float32)    
    for ind,batch in enumerate(X):        
        for j, elem in enumerate(batch):            
            x[ind, j, elem] = 1            
    return x


In [68]:

# Generator function to generate infinite-stream of inputs for training

def generate_batch(batch_size=32, seq_len=10, max_no=100):
    # Randomly generate a batch of integer sequences (X) and its sorted counterpart (Y)
    x = np.zeros((batch_size, seq_len, max_no), dtype=np.float32)    
    y = np.zeros((batch_size, seq_len, max_no), dtype=np.float32)    
    while True:
	# Generates a batch of input
        X = np.random.randint(max_no, size=(batch_size, seq_len))        
        Y = np.sort(X, axis=1)        

        for ind,batch in enumerate(X):            
            for j, elem in enumerate(batch):                
                x[ind, j, elem] = 1

        for ind,batch in enumerate(Y):
            for j, elem in enumerate(batch):
                y[ind, j, elem] = 1
        
        yield x, y        
        x.fill(0.0)
        y.fill(0.0)

In [69]:
def create_model(seq_len, max_no, n_layers, hidden_size):
    model = Sequential()
    # the encoder LSTM
    model.add(LSTM(hidden_size, input_shape=(seq_len, max_no)))
    # in next layer, repeat the input seq_len times
    model.add(RepeatVector(seq_len))
    # decoder RNN, which will return output sequence
    for _ in range(n_layers):
        model.add(LSTM(hidden_size, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(max_no)))
    model.add(Activation('softmax'))
    return model

In [70]:
# Global parameters.
batch_size=32
seq_len =  15 # number of elements in sequence to sort
max_no = 100 # upper range of the numbers in sequence
hidden_size = 128
n_layers = 2
epochs = 100000

model = create_model(seq_len, max_no, n_layers, hidden_size)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [71]:
# Now the training loop, we'll sample input batches from the generator function written previously and feed it to the RNN for learning

for ind,(X,Y) in enumerate(generate_batch(batch_size, seq_len, max_no)):
    loss, acc = model.train_on_batch(X, Y)
    # We'll test RNN after each 500 iteration to check how well it is performing
    if ind % 500 == 0:
        testX = np.random.randint(max_no, size=(1, seq_len))
        test = one_hot_encode(testX, seq_len, max_no)        
        print(ind, ': epochs')
        print(acc, ': accuracy')
        loss_str = '{:4.3}'.format(loss)
        acc_str = '{:4.3}'.format(acc)
        
        y = model.predict(test, batch_size=1)
        np_sorted = np.sort(testX)[0]
        lstm_sorted = np.argmax(y, axis=2)[0]
        is_equal = np.array_equal(np_sorted, rnn_sorted)
        print('Expexted Output : ', np_sorted)
        print('Actual Output   :', lstm_sorted)
        print("\n")
        
        if acc > 0.995 :
            model.save('/content/gdrive/My Drive/Colab Notebooks/sorter-model-{}.h5'.format(ind))
            break
        
        
    if ind > epochs:
        # Save trained model
        # model.save('model.h5')         
        model.save('/content/gdrive/My Drive/Colab Notebooks/sorter-model.h5')
        break

0 : epochs
0.01666666753590107 : accuracy
Expexted Output :  [ 4  8 10 19 20 33 34 42 47 52 75 77 78 91 97]
Actual Output   : [37 37 37 37 37 37 37 37 37 37 37 37 37 37 37]


500 : epochs
0.09583333134651184 : accuracy
Expexted Output :  [12 16 20 21 21 26 30 32 46 58 63 70 73 78 78]
Actual Output   : [ 0  4 12 12 20 26 29 41 45 57 61 69 72 78 84]


1000 : epochs
0.14791665971279144 : accuracy
Expexted Output :  [28 30 32 52 53 59 70 72 75 82 84 84 89 92 95]
Actual Output   : [23 31 39 52 56 63 70 75 79 81 85 87 88 93 97]


1500 : epochs
0.15833333134651184 : accuracy
Expexted Output :  [ 1 13 16 26 26 31 35 52 61 65 71 75 84 86 99]
Actual Output   : [ 1 10 16 23 27 30 33 54 59 64 68 75 83 87 93]


2000 : epochs
0.18958333134651184 : accuracy
Expexted Output :  [ 6  9 14 23 27 37 41 62 63 65 73 85 87 92 96]
Actual Output   : [ 0 10 15 22 29 36 41 60 63 65 72 84 88 91 94]


2500 : epochs
0.23125000298023224 : accuracy
Expexted Output :  [11 27 35 47 50 54 60 63 64 67 72 86 96 97 98]
Act

# New Section